# JWST Duplication Checking

## Introduction

As with _HST, JWST_ observations that duplicate existing, planned, or approved observations will be allowed only if investigators provide a scientific justification in their proposal and that request is approved. Consult the [JWST Duplicate Observations Policy](https://jwst-docs.stsci.edu/jwst-opportunities-and-policies/jwst-general-science-policies/jwst-duplicate-observations-policy) for details. Broadly speaking, observations you are considering might duplicate observations in the current or prior cycles if 
your target is of the same astrophysical source, or there is significant spatial overlap of fields, **and** the following apply:

  * Similar imaging passband, or overlapping spectral range
  * Similar (spectral) resolution
  * Similar exposure depth

Observations with a different scientific instrument than one you are considering *might* still duplicate if the observing configuration and capabilities are similar (e.g., NIRCam and NIRISS imaging, or NIRCam and NIRSpec spectroscopy).

This notebook illustrates how to use the python package [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) to search the Mikulski Archive for Space Telescopes (MAST) for potential duplicate observations. It is also possible to perform these searches with the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html), but often less efficiently. 

### Special Disclaimer
<a id="Disclaimer"></a>

<div class="alert alert-block alert-warning">

<span style="color:black">
The complete footprint of approved (but not executed) mosaicked or parallel observations is only approximate. That is, while the primary location is reported for an observation, the exact orientation. of the aperture is not known until the observation is scheduled for execution. Moreover metadata in MAST about planned/approved observations may not suffice to determine whether your intended observation is a genuine duplication, particularly for slit or MOS spectroscopy. You are responsible for evaluating the details of the planned observations by using the accepted program's APT file (and/or the Aladin display in APT, as appropriate) to determine if the potential duplications are genuine.
</span>
</div>


### Table of contents

<ul>
    <li><a href="#Strategy">Strategy</a></li>
    <li><a href="#Setup">Setup</a></li>
    <ul>
        <li><a href="#Utility Routines">Utility Routines</a></li>
    </ul>
    <li><a href="#Target/Position Queries">Queries by Target/Position</a></li>
    <ul>
        <li><a href="#Target Search">Search by Target Name</a></li>
        <li><a href="#Moving-Target">Single Moving Target</a></li>
    </ul>
    <li><a href="#Target-List">Checking a List of Targets</a></li>
    <ul>
        <li><a href="#Target-CSV">Loading Targets from CSV</a></li>
    </ul>
    <li><a href="#Evaluate-Dups">Evaluating Potential Duplications</a></li>
    <ul>
        <li><a href="#Trappist-1">Trappist-1</a></li>
        <li><a href="#V-XX-Cha">V* XX Cha</a></li>
        <li><a href="#M-57">M 57</a></li>
    </ul>
    <li><a href="#Appendix">Appendix: Caveats</a></li>
    <ul>
        <li><a href="#Source">Source Name</a></li>
        <li><a href="#Search">Cone Search vs. Area Search</a></li>
    </ul>
    <li><a href="#Resources">Additional Resources</a></li>
</ul>

<a id="Strategy"></a>
## Strategy

The following strategy, applied to each target or field, will identify potentially duplicative observations. 

  1. Search for common targets or fields. If no existing or planned observation coincides with your intended target, you are done.
  2. If there is a spatial overlap, determine if the instrument(s) and observing configurations you plan to use are the same or similar to the existing observations. If there is no commonality, you are done.
  3. If there is spatial overlap and common instruments/configuration, determine in detail the overlap in passband/spectral coverage, and exposure depth.
  4. If there is a likely duplication for an intended target, do one of the following:
     - Include in your proposal a scientific case for the duplicating observation(s)
     - Alter your intended observation(s) in a way that does not duplicate
     - Choose a different target

<a id="Setup"></a>
## Setup

Begin by importing some essential python packages: general utilities in [astropy](https://www.astropy.org/), and query services in [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html). 

In [1]:
# Give the notebook cells more of the available width
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
import astropy
from astropy import table
from astropy.table import Table, unique
from astropy import units as u
from astropy.coordinates import Angle
from astroquery.mast import Mast
from astroquery.mast import Observations

<a id="Utility Routines"></a>
### Utility Routines

The following utility routine will create URLs to the parent programs of matching observations. 

In [3]:
APT_LINK = '(http://www.stsci.edu/cgi-bin/get-proposal-info?id={}&observatory=JWST)'

def get_program_URL(program_id):
    '''
    Generate the URL for program status information, given a program ID. 
    '''
    return APT_LINK.format(program_id)

<a id="Target/Position Queries"></a>
## Queries by Target/Postion

All of the queries below search for JWST observations, using a search radius larger than fields of view (FoV) of JWST apertures, to allow for the possibility that the FoV may be rotated when approved-but-unexecuted observations are actually scheduled. Your queries will typically be more efficient if you restrict your search to JWST data by including the parameter: `obs_collection = "JWST"`. 

<div class="alert alert-block alert-warning">

<span style="color:black">
<b>Searching for source names is not recommended</b> when checking for potential duplications with fixed targets because the matches are not reliable, consistent, or complete. <i>The exception would be for matching Object names of well known solar system bodies.</i>

The best approach is to use an independent source for the coordinates of your desired target, including existing JWST or HST images if any exist. Failing that, the <b>Mast.resolve_object()</b> method will return coordinates in the ICRS reference frame, which you can use to verify whether a target name resolves to the coordinates you intend. See the <a href="#Source">Appendix</a> for details.
</span>
</div>

<a id="Target Search"></a>
### Search by Target Name

This example shows how to query for a single target with a standard name, **HD 104237** which is a T-Tauri star. We first resolve the name and verify that the coordinates are correct, using the `Mast` class and the `resolve_object()` method. 

In [4]:
# Resolve the coordinates if necessary
coords = Mast.resolve_object('HD 104237')
print(coords)

<SkyCoord (ICRS): (ra, dec) in deg
    (180.02119525, -78.19293492)>


In this case the resolved coordinates agree with those in [Simbad](https://simbad.u-strasbg.fr/simbad/sim-basic?Ident=HD+104237&submit=SIMBAD+search), so it is safe to execute a simple **cone search** (a search within a specified radius of a place on the sky) to find JWST existing or planned observations. 
The [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) package provides the method **Observations.query_criteria()** to specify the parameters for the search; provide them as `key=value` pairs. The full set of query parameters for this method may be found on [CAOM Field Descriptions](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). 

In [5]:
# If the coordinates are ok
obs = Observations.query_criteria(
        obs_collection='JWST'
        ,objectname='HD 104237'
        ,radius='30s'
        )
print('Number of matching observations: {}'.format(len(obs)))

Number of matching observations: 0


<div class="alert alert-block alert-info">

<span style="color:black">
    If the number of matches is zero, there are no potential JWST duplications with this target.
</span>
</div>

<a id="Moving-Target"></a>
### Single Moving Target

Moving targets, by definition, do not lend themselves to searches by position. This kind of search in MAST is limited to a modest set of solar system bodies with recognized names. Note the use of a wildcard character (*) to trigger a match, even when the target name includes other text.

In [6]:
obs = Observations.query_criteria(
        obs_collection="JWST"
        ,target_name="Io*"
        )
    
print(f'Number of matching observations: {len(obs)}')

Number of matching observations: 34


There are several JWST observations of **Io**, including some that are planned but have not executed as of the beginning of Cy-2 (i.e., `calib_level = -1`). The details are in the returned table of results, the most essential of which can be viewed below.

In [7]:
out_cols = ['target_name','instrument_name','filters','calib_level','t_exptime','proposal_id']
obs[out_cols].show_in_notebook()

idx,target_name,instrument_name,filters,calib_level,t_exptime,proposal_id
0,IO,MIRI/IFU,--,-1,2664.038,4565
1,IO,MIRI/IFU,--,-1,222.003,4565
2,IO,MIRI/IFU,--,-1,222.003,4565
3,IO,MIRI/IFU,--,-1,222.003,4565
4,IO,MIRI/IFU,--,-1,222.003,4565
5,IO,MIRI/IFU,--,-1,222.003,4565
6,IO,MIRI/IFU,--,-1,222.003,4565
7,IO,MIRI/IFU,--,-1,222.003,4565
8,IO,MIRI/IFU,--,-1,222.003,4565
9,IO,NIRSPEC/IFU,F290LP;G395H,3,343.576,1373


You may need to examine the specifics of the programs that obtained the observations to know whether your intended observations would duplicate. Here extend the results table with the unique Program Titles and URLs for the program status pages, which may offer clues:

In [8]:
obs['title'] = [x[:80] for x in obs['obs_title']]
unique(obs['proposal_id','title']).pprint(max_width=-1)
obs['proposal_URL'] = [get_program_URL(x) for x in obs['proposal_id']]
unique(obs['proposal_id','proposal_URL']).pprint(max_width=-1)

proposal_id                                      title                                      
----------- --------------------------------------------------------------------------------
       1373 ERS observations of the Jovian System as a demonstration of JWST capabilities fo
       4078 Mass-loss from Ios volcanic atmosphere: A unique synergy with the Juno Io fly-by
       4565              HST-Juno Io Campaign: Connecting Volcanos to the Plasma Environment
proposal_id                                proposal_URL                              
----------- -------------------------------------------------------------------------
       1373 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1373&observatory=JWST)
       4078 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=4078&observatory=JWST)
       4565 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=4565&observatory=JWST)


<div class="alert alert-block alert-warning">

<span style="color:black">
    There are images, spectra, coronagraphic, and IFU observations of this target that could conflict with your intended proposal. Proposed images in passbands that overlap the IFU data would have to be examined closely for duplication. 
</span>
</div>

<a id="Target-List"></a>
## Checking a List of Targets

It is often useful to search for individual targets with the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) because the results are easily visualized. But it is more efficient to search over a large number of targets using astroquery.mast. 

<a id="Target-CSV"></a>
### Loading Targets from CSV
For efficiency, make a CSV (comma-separated variable) list of your targets, one per row. The list can alternatively be read from a local file: just substitute the file name for `targ_table` in the first argument of the `Table.read()` method. The list can contain many fields, but at a minimum must contain the target name and the equitorial coordinates. 

<div class="alert alert-block alert-info">

<span style="color:black">
For stellar targets, it is best to use a large enough radius to make your search robust against position uncertainties and proper motion between J2000 and the current epoch.
</span>
</div>

The *first row of the file* will be interpreted as a column name in the table. <b>This is important.</b> 

In [9]:
targ_table = '''
target_name, RA, DEC, Radius, Description
Trappist-1, 23:06:29.3684948589, -05:02:29.037301866, 30s, Exoplanet host star
V* XX Cha, 11:11:39.559, -76:20:15.04, 30s, Variable star with debris disk
M 31, 00:42:44.330, +41:16:07.50, 12.0m, Andromeda Galaxy
M 57, 18:53:35.0967659112, +33:01:44.883287544, 2m, Planetary nebula
'''
targets = Table.read(targ_table, format='ascii.csv')

Next, make new columns to hold the coordinates and ranges in units of degrees. 

In [10]:
targets['ra_deg'] = [Angle(x+' hours').degree for x in targets['RA']]
targets['dec_deg'] = [Angle(x+' degree').degree for x in targets['DEC']]
targets['radius_deg'] = [Angle(x).degree for x in targets['Radius']]
targets['N_obs'] = 0  # field to hold the count of matched observations 
targets

target_name,RA,DEC,Radius,Description,ra_deg,dec_deg,radius_deg,N_obs
str10,str19,str19,str5,str30,float64,float64,float64,int64
Trappist-1,23:06:29.3684948589,-05:02:29.037301866,30s,Exoplanet host star,346.62236872857875,-5.041399250518333,0.008333333333333333,0
V* XX Cha,11:11:39.559,-76:20:15.04,30s,Variable star with debris disk,167.91482916666666,-76.33751111111111,0.008333333333333333,0
M 31,00:42:44.330,+41:16:07.50,12.0m,Andromeda Galaxy,10.684708333333331,41.26875,0.2,0
M 57,18:53:35.0967659112,+33:01:44.883287544,2m,Planetary nebula,283.39623652462996,33.029134246539996,0.03333333333333333,0


It can save some effort to determine the number of JWST Observations that match each of your targets using the <b><code>query_criteria_count()</code></b> method. Note that the coordinate parameters must be specified as lower and upper bounds of a range (i.e., a python list), so use the `coord_ranges()` utility function.

<div class="alert alert-block alert-info">

<span style="color:black">
It is good practice to first check the number of matching observations before fetching the results themselves, in case the number of results is very large. This is more important when querying MAST missions with a very large number of observations, such as <i>HST</i>. This type of search is typically fairly fast.
    
</span>
</div>

In [11]:
# get the counts of each target
for t in targets: 
    t['N_obs'] = Observations.query_criteria_count(
        obs_collection='JWST'
        ,coordinates=f"{t['ra_deg']} {t['dec_deg']}"
        ,radius=t['radius_deg']
        )
targets['target_name','N_obs'].show_in_notebook()

idx,target_name,N_obs
0,Trappist-1,304
1,V* XX Cha,14
2,M 31,0
3,M 57,143


<div class="alert alert-block alert-info">

<span style="color:black">
For targets where no JWST Observations exist, there is no potential for duplication. 
</span>
</div>

<a id="Evaluate-Dups"></a>
## Evaluating Potential Duplications

To examine the matching Observations for the other targets in more detail, use the <b><code>query_criteria()</code></b> method. Also, use a **cone search** (i.e., specify a coordinate pair and a radius), rather than an area search over a region in RA and Dec; see the <a href="#Search">Appendix</a> for details. Note that a new column called "obs" is first added to the table to contain the results of the Observation search. 

In [12]:
targets['obs'] = None
for t in targets: 
    t['obs'] = Observations.query_criteria(
        obs_collection='JWST'
        ,coordinates=f"{t['ra_deg']} {t['dec_deg']}"
        ,radius=t['radius_deg']
        )

Targets with matching JWST Observations are examined in the following sub-sections.

<a id="Trappist-1"></a>
### Trappist-1

Trappist-1 is a well known exo-planet host star. If the intended observations are timeseries spectroscopy, the search would naturally be limited to a small area of sky. There are a several *unique* JWST observations of this target, so it is important to examine the relevant details. 

In [13]:
out_cols = ['target_name','instrument_name','filters','t_exptime','proposal_id']
obs = targets[0]['obs']
unique(obs[out_cols]).show_in_notebook()

idx,target_name,instrument_name,filters,t_exptime,proposal_id
0,TRAPPIST-1,MIRI/IMAGE,F1280W,12869.176,5191
1,TRAPPIST-1,MIRI/IMAGE,F1280W,14819.052,5191
2,TRAPPIST-1,MIRI/IMAGE,F1280W,16546.084,5191
3,TRAPPIST-1,MIRI/IMAGE,F1280W,16741.071,5191
4,TRAPPIST-1,MIRI/IMAGE,F1500W,11535.841,2304
5,TRAPPIST-1,MIRI/IMAGE,F1500W,11574.692,2304
6,TRAPPIST-1,MIRI/IMAGE,F1500W,74151.9,3077
7,TRAPPIST-1,MIRI/IMAGE,F1500W,138234.537,3077
8,TRAPPIST-1,NIRISS/SOSS,CLEAR;GR700XD,988.92,2589
9,TRAPPIST-1,NIRISS/SOSS,CLEAR;GR700XD,11965.932,1201


You will need to examine the specifics of the programs that obtained the observations to know for certain which of the 7 known exoplanets was being observed, and therefore whether your intended observations would duplicate. Here are the unique Program Titles, which may offer clues:

In [14]:
obs['title'] = [x[:80] for x in obs['obs_title']]
unique(obs['proposal_id','title']).pprint(max_width=-1)

proposal_id                                      title                                      
----------- --------------------------------------------------------------------------------
       1177                                       MIRI observations of transiting exoplanets
       1201  NIRISS Exploration of the Atmospheric diversity of Transiting exoplanets (NEAT)
       1279                                               Thermal emission from Trappist-1 b
       1331                                              Transit Spectroscopy of TRAPPIST-1e
       1981 Tell Me How Im Supposed To Breathe With No Air: Measuring the Prevalence and Div
       2304                  Hot Take on a Cool World:  Does Trappist-1c Have an Atmosphere?
       2420     Probing the Terrestrial Planet TRAPPIST-1c for the Presence of an Atmosphere
       2589                             Atmospheric reconnaissance of the TRAPPIST-1 planets
       2742                                       NIRSpec darks for de

The Program Status Pages offer the complete specifications for each existing program:

In [15]:
obs['proposal_URL'] = [get_program_URL(x) for x in obs['proposal_id']]
unique(obs['proposal_id','proposal_URL']).pprint(max_width=-1)

proposal_id                                proposal_URL                              
----------- -------------------------------------------------------------------------
       1177 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1177&observatory=JWST)
       1201 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1201&observatory=JWST)
       1279 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1279&observatory=JWST)
       1331 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1331&observatory=JWST)
       1981 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=1981&observatory=JWST)
       2304 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=2304&observatory=JWST)
       2420 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=2420&observatory=JWST)
       2589 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=2589&observatory=JWST)
       2742 (http://www.stsci.edu/cgi-bin/get-proposal-info?id=2742&observatory=JWST)
       3077 (http://www.stsci.edu/cgi-bin/get-proposal

<div class="alert alert-block alert-warning">

<span style="color:black">
    There are MIRI imaging observations (likely Time-series) in multiple bands, and both NIRISS and NIRSpec spectroscopic observations of this target that could conflict with your intended proposal. 
</span>
</div>

<a id="V-XX-Cha"></a>
### V* XX Cha

This pre-main-sequence star has a debris disk, and might be worth a coronagraphic observation. The search area is small (20 arcsec radius) to exclude nearby targets.

In [16]:
out_cols = ['target_name','instrument_name','filters','t_exptime','calib_level','proposal_id']
obs = targets[1]['obs']
unique(obs[out_cols]).show_in_notebook()

idx,target_name,instrument_name,filters,t_exptime,calib_level,proposal_id
0,V-XX-CHA-2,MIRI/IFU,CH1,3696.348,3,1282
1,V-XX-CHA-2,MIRI/IFU,CH2,3696.348,3,1282
2,V-XX-CHA-2,MIRI/IFU,CH3,3696.348,3,1282
3,V-XX-CHA-2,MIRI/IFU,CH4,3696.348,3,1282
4,V-XX-CHA-2,MIRI/IMAGE,F1280W,3696.348,3,1282
5,V-XX-CHA-2,NIRSPEC/IFU,F290LP;G395H,32.21,2,1282
6,V-XX-CHA-2,NIRSPEC/IFU,F290LP;G395H,1288.416,3,1282


<div class="alert alert-block alert-info">

<span style="color:black">
While JWST spectra from MIRI and NIRSpec exist, there is no potential for duplication with a coronagraphic observation. 
</span>
</div>

<a id="M-57"></a>
### M 57

The Ring Nebula (NGC 6720) is well observed, particularly with HST and, recently, with JWST. We searched a 4-arcmin region around the source to include potential spectral Observations in the nebular periphery. The following table shows the unique combinations of instrument configurations. 

In [17]:
out_cols = ['target_name','instrument_name','filters','t_exptime','proposal_id']
obs = targets[3]['obs']
unique(obs[out_cols]).show_in_notebook()

idx,target_name,instrument_name,filters,t_exptime,proposal_id
0,NGC6720,MIRI/IMAGE,F1000W,444.0079999999999,1558
1,NGC6720,MIRI/IMAGE,F1130W,444.0079999999999,1558
2,NGC6720,MIRI/IMAGE,F1280W,444.0079999999999,1558
3,NGC6720,MIRI/IMAGE,F1500W,444.0079999999999,1558
4,NGC6720,MIRI/IMAGE,F1800W,444.0079999999999,1558
5,NGC6720,MIRI/IMAGE,F2100W,444.0079999999999,1558
6,NGC6720,MIRI/IMAGE,F2550W,444.0079999999999,1558
7,NGC6720,MIRI/IMAGE,F560W,444.0079999999999,1558
8,NGC6720,MIRI/IMAGE,F770W,444.0079999999999,1558
9,NGC6720,NIRCAM/IMAGE,F150W2;F162M,483.15599999999984,1558


<div class="alert alert-block alert-warning">

<span style="color:black">
    There are both MIRI and NIRCam images in many passbands, and MIRI and NIRSpec spectroscopic IFU Observations of this target that could conflict with the intended proposal. 
</span>
</div>

To retain this target it would be necessary to obtain spectra in a different location, or to use a different instrument/configuration/data-taking mode (e.g., time-series of the central star). It may be helpful to see the [footprints of these Observations](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html?searchQuery=%7B%22service%22%3A%22CAOMFILTERED%22%2C%22inputText%22%3A%5B%7B%22paramName%22%3A%22obs_collection%22%2C%22niceName%22%3A%22obs_collection%22%2C%22values%22%3A%5B%22JWST%22%5D%2C%22valString%22%3A%22JWST%22%2C%22isDate%22%3Afalse%2C%22separator%22%3A%22%3B%22%2C%22facetType%22%3A%22discrete%22%2C%22displayString%22%3A%22JWST%22%7D%5D%2C%22position%22%3A%22283.39623652463%2C%2033.02913424654%2C%200.066667%22%2C%22paramsService%22%3A%22Mast.Caom.Filtered.Position%22%2C%22title%22%3A%22MAST%3A%20%20Advanced%20Search%201%22%2C%22tooltip%22%3A%22JWST%3B%20%20m%2057%20r%3D4m%3B%22%2C%22ra%22%3A283.39623652463%2C%22dec%22%3A33.02913424654%2C%22radius%22%3A0.066667%2C%22columns%22%3A%22*%22%2C%22columnsConfig%22%3A%22Mast.Caom.Cone%22%7D) in the MAST Portal, or to view the observation specifications in Program [GO-1558](https://www.stsci.edu/cgi-bin/get-proposal-info?id=1558&observatory=JWST).

<a id="Appendix"></a>
## Appendix: Caveats
<a id="Source"></a>
### Source Name

It is possible to search MAST for individual sources by name, in one of two ways: *Object Name* or by *Target Name*. 

- **Object Name:** MAST invokes an astrophysical name resolver (e.g., [NED](https://ned.ipac.caltech.edu/byname) or [Simbad](https://simbad.u-strasbg.fr/simbad/sim-fid)) to look up source coordinates. Not all object names are recognized by resolvers. Also, different resolvers sometimes return substantially different coordinates; MAST will pick one of them (but not tell you which one). Sometimes the coordinates returned by the resolvers differ by substantial amounts, often because they refer to different sources.
  - A classic case is the bright star &mu; Eri from the Bayer Greek letter system, and the star MU Eri in the general catalog of variable stars. in MAST,
    - Searching for `* mu. eri` (or just plain `mu eri` matches **&mu; Eri** (RA = 4:45:30.15, Dec = -3:15:16.777)
    - Searching for `V* mu eri` matches **MU Eri** (RA = 2:48:10.566, Dec = -15:18:04.03)
- **Target Name:** This search matches to target names specified by Investigators in observing proposals to refer to sources. These are not guaranteed to match standard names for astrophysical sources, and often do not.

<div class="alert alert-block alert-warning">

<span style="color:black">
<b>Searching by source names is not recommended</b> when checking for potential duplications with fixed targets because the matches are not reliable, consistent, or complete. <i>The exception would be for matching Object names of well known solar system bodies.</i>

The best approach is to use an independent source for the coordinates of your desired target. Failing that, the <b>Mast.resolve_object()</b> method will return coordinates in the ICRS reference frame, which you can use to verify whether a target name resolves to the coordinates you intend. 
</span>
</div>

<a id="Search"></a>
### Cone Search vs Area Search

MAST offers two basic kinds of spatial search: 
 - **cone**: a region of sky defined by a pair of coordinates and a search radius (it is a "cone" in the sense that the area of the search expands as the source distance increases). 
 - **area**: a region of sky bounded by ranges in each of the equitorial coordinates, RA and Dec.

These searches use different methods to identify overlaps with Observations in MAST. The **cone search** matches if *any part* of the observation aperture intersects any part of the cone. The **area search** matches only if *the center of the observation aperture* lies within the search region. 

<div class="alert alert-block alert-warning">

<span style="color:black">
<b>Searching by area is not recommended</b> when checking for potential duplications with fixed targets because the matches may not include genuine overlaps of existing or planned observations.

The best approach is to use a <b>cone search</b> centered on the coordinates of your desired target.
</span>
</div>

<a id="Resources"></a>
## Additional Resources

* [astropy](https://docs.astropy.org/en/stable/index.html) documentation
* [astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html) documentation for querying MAST
* [Queryable fields](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html) in the MAST/CAOM database
* The [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) web interface

## About this notebook

This notebook was developed by Archive Sciences Branch staff. For support, please direct questions to the Archive HelpDesk at archive@stsci.edu, or through the [JWST HelpDesk Portal](https://jwsthelp.stsci.edu). 
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

**Last update:** 2024 Jan